In [1]:
from pyhmy import cli
import json
import pandas as pd
import numpy as np

In [2]:
env = cli.download()
cli.environment.update(env)

Saved harmony binary to: `/Users/yishuang/Desktop/log_analysis/notebooks/test_log_analysis/bin/hmy`


In [3]:
path = os.getcwd() + "/bin/hmy"
cli.set_binary(path)

True

In [4]:
validator = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator all"))['result']   

In [5]:
shard = []
availability = []
name = []
status = []
website = []
details = []
address = []
for i in validator:  
    try:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        if info['current-epoch-voting-power']:
            shard.append(info['current-epoch-voting-power'][0]['shard-id'])
        else:
            shard.append(np.nan)
        availability.append(info['validator']['availability'])
        name.append(info['validator']['name'])
        status.append(info['validator']["epos-eligibility-status"])
        website.append(info['validator']['website'])
        details.append(info['validator']['details'])
        address.append(info['validator']['address'])
    except:
        print(i)

In [6]:
df = pd.DataFrame(list(zip(shard, availability, name, status, website, details, address)), \
                  columns =['shard-id', 'availability', 'name','status','website','details','address'])
df = pd.concat([df.drop(['availability'], axis=1), df['availability'].apply(pd.Series)], axis=1)

In [7]:
df = df.sort_values(by=['shard-id'])
df.reset_index(inplace = True, drop = True)

In [8]:
df

,shard-id,name,status,website,details,address,num-blocks-signed,num-blocks-to-sign
0,0,OSTNSentry0,active,sentry@harmony.one,OSTN Sentry for Shard 0,one1m6j80t6rhc3ypaumtsfmqwjwp0mrqk9ff50prh,2220,2355
1,0,Bubz,inactive,https://i.imgur.com/G6Mruca.jpg,https://www.youtube.com/watch?v=QQNL83fhWJU,one1z52vrx2tta07lk6pww2vgtc9wt2jws3scvh9uh,0,75
2,0,Bubz,inactive,https://i.imgur.com/G6Mruca.jpg,https://www.youtube.com/watch?v=QQNL83fhWJU,one13v3ar3gpfx5f34gkwac229m59h0amr9lvnnuay,0,75
3,0,EV14_Validator,inactive,https://i.imgur.com/G6Mruca.jpg,https://www.youtube.com/watch?v=QQNL83fhWJU,one1x3egg0tqqljylsflzyzxy3lt4qgakag80s9up2,0,75
4,0,Bubz,inactive,https://i.imgur.com/G6Mruca.jpg,https://www.youtube.com/watch?v=QQNL83fhWJU,one1njrpkgs6n5fza67cg6p8g78xedr7mmjrd76zpg,0,75
...,...,...,...,...,...,...,...,...
83,3,OSTNSentry3,active,sentry@harmony.one,OSTN Sentry for Shard 3,one1marnnvc8hywmfxhrc8mtpjkvvdt32x9kxtwkvv,2438,2464
84,3,Sesameseed,active,https://sesameseed.org/,Sesameseed is a multi-chain digital asset stak...,one1a8avzz3hcvhfrh2tp7rdprpvwt838y9um9c2q7,1758,1934
85,3,gondar,active,github.com,Bounty Hunter,one1hpxxxnqp5epvs2ktzdft24q39r4pttywkmt3cy,1857,1875
86,3,Vega,active,https://giphy.com/gifs/rick-roll-lgcUUCXgC8mEo,Cheap Stake here,one1wz9fmjrwua3le0c8qxv058p3wpdg4ctjyc3ha4,1788,1801


In [9]:
df.to_csv("../../csv/validator.tsv", index=False)